In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
base_path = "/content/drive/My Drive/yl_tez"
cihaz = "cuda"

In [3]:
import os
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Yerelde Çalışıyor.")
    base_path = "."

Mounted at /content/drive


In [4]:
model_adi = "BestModelParameters"
model_yolu = os.path.join(base_path, model_adi)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_yolu)
model = AutoModelForCausalLM.from_pretrained(model_yolu)
model.to(cihaz)
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50262, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50262, bias=False)
)

In [6]:
import ipywidgets as widgets
from IPython.display import display, HTML

In [7]:
def generate_text(full_prompt_text, tokenizer, model, device, max_length=450):
    # Girdi metnini tokenize et ve attention mask oluştur
    encoded_input = tokenizer.encode(full_prompt_text, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)

    # Metin üretimi
    output_sequences = model.generate(
        input_ids=encoded_input,
        max_length=max_length,  # Cevap için ekstra uzunluk
        min_length=50,  # Minimum uzunluk belirleme
        temperature=0.7,  # Rastgelelik parametresi
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    # Cevabı girdiden ayırma
    generated_response = generated_text[len(full_prompt_text):].strip()

    return generated_response
def apply_chat_template(message, doctor_title, doctor_speciality):
    user_message = f"<|USER|> {message}\n"
    doctor_info = f"<|DOCTOR_TITLE|> {doctor_title} <|SPECIALITY|> {doctor_speciality}\n"
    assistant_message = f"<|ASSISTANT|>"
    return f"{user_message}\n{doctor_info}\n{assistant_message}"

def test_et(tokenizer, model, device, titles, specialities):
    while True:

        speciality_dropdown = widgets.Dropdown(
            options=specialities,
            description='Uzmanlık alanı:',
            disabled=False,
        )
        display(speciality_dropdown)
        title_dropdown = widgets.Dropdown(
            options=titles,
            description='Doktor unvanı:',
            disabled=False,
        )
        display(title_dropdown)

        # Kullanıcıdan soru girdisi alma
        prompt_text = input("Soru: \n")
        if prompt_text == '0':
            break
        speciality_dropdown.disabled = True
        title_dropdown.disabled = True
        selected_title = title_dropdown.value
        selected_speciality = speciality_dropdown.value

        # Tam soru metnini oluştur
        full_prompt_text = apply_chat_template(prompt_text, selected_title, selected_speciality)

        # Metin üretimi ve çıktının gösterilmesi
        generated_text = generate_text(full_prompt_text, tokenizer, model, device)
        display(HTML(f"<strong>{generated_text}</strong>"))


# Ünvan Alan Çiftleri

In [8]:
titles = [
"Dr. Dt.",
"Prof. Dr. Dt.",
"Uzm. Dyt.",
"Dr. Psk.",
"Uzm. Psk.",
"Dr.",
"Psk.",
"Dil ve Konuşma Terapisti",
"Uzm. Psk. Dan.",
"Dr. Öğr. Üyesi Dt.",
"Klinik Psikolog ",
"Dr. Dyt. ",
"Yrd. Doç. Dr.",
"Uzm. Dr. Dt.",
"Çocuk Gelişim Uzmanı",
"Doç. Dr. Psk. Dan",
"Aile Danışmanı",
"Pedagog",
"Uzm. Kl. Psk.",
"Uzm. Dr.",
"Psikoterapist",
"Uzm. Dt.",
"Op. Dr.",
"Doç. Dr. Dt.",
"Ergoterapist",
"Dr. Psk. Dan.",
"Prof. Dr.",
"Dt.",
"Fzt.",
"Dr. Öğr. Üyesi",
"Doç. Dr.",
"Dyt.",
"Vet.",
"Dr.Öğr.Üyesi",
"Psk. Dan.",
"Veteriner Hekim"
]
titles = sorted(titles)

specialities = [
"cocuk-sagligi-ve-hastaliklari-cocuk-genetik-hastaliklari",
"acil-tip",
"cocuk-gelisim-uzmani",
"pratisyen-hekimlik",
"cocuk-sagligi-ve-hastaliklari-cocuk-gogus-hastaliklari",
"aile-danismani",
"genel-cerrahi-endokrin-cerrahisi",
"histoloji-ve-embriyoloji",
"kadin-hastaliklari-ve-dogum-ureme-endokrinolojisi-ve-infertilite",
"aile-hekimligi",
"fiziksel-tip-ve-rehabilitasyon-fiziksel-tip-ve-rehabilitasyon-romatoloji",
"cocuk-sagligi-ve-hastaliklari-cocuk-nefrolojisi",
"romatoloji-dahiliye-ve-ic-hastaliklari",
"uroloji",
"enfeksiyon-hastaliklari-ve-klinik-mikrobiyoloji",
"tibbi-biyokimya",
"dis-hekimi-koruyucu-dis-hekimlig",
"radyasyon-onkolojisi",
"cocuk-cerrahisi-cocuk-cerrahisi-cocuk-urolojisi-cocuk-urolojisi-uroloji",
"genel-cerrahi-proktoloji",
"cocuk-sagligi-ve-hastaliklari-cocuk-hematoloji",
"dis-hekimi-restoratif-dis-tedavisi",
"fizyoterapi",
"cocuk-cerrahisi",
"ortopedi-ve-travmatoloji",
"tibbi-patoloji",
"fiziksel-tip-ve-rehabilitasyon-fiziksel-tip-ve-rehabilitasyon-algoloji",
"kadin-hastaliklari-ve-dogum-perinatoloji",
"dahiliye-ve-ic-hastaliklari-hematoloji",
"genel-cerrahi-cerrahi-onkoloji",
"dahiliye-ve-ic-hastaliklari-gastroenteroloji",
"dahiliye-ve-ic-hastaliklari-immunoloji",
"kalp-damar-cerrahisi-damar-cerrahisi",
"radyoloji-girisimsel-radyoloji",
"cocuk-sagligi-ve-hastaliklari-cocuk-immunolojisi-ve-alerji-hastaliklari",
"ergoterapi",
"veteriner",
"cocuk-sagligi-ve-hastaliklari-cocuk-romatolojisi",
"dermatoloji",
"cocuk-sagligi-ve-hastaliklari",
"psikolojik-danisman-psikoloji",
"geleneksel-ve-tamamlayici-tip",
"cocuk-cerrahisi-cocuk-cerrahisi-cocuk-urolojisi",
"dahiliye-ve-ic-hastaliklari-nefroloji",
"cocuk-sagligi-ve-hastaliklari-cocuk-enfeksiyon-hastaliklari",
"noroloji-klinik-norofizyoloji",
"klinik-psikolog-psikoloji",
"psikolojik-danisman",
"psikoterapi",
"dahiliye-ve-ic-hastaliklari",
"dis-hekimi-oral-implantoloji",
"kadin-hastaliklari-ve-dogum-jinekolojik-onkoloji",
"uroloji-androloji",
"genel-cerrahi-meme-cerrahisi",
"cocuk-sagligi-ve-hastaliklari-cocuk-kardiyolojisi",
"genel-cerrahi-gastroenteroloji-cerrahisi",
"dis-hekimi-ortodonti",
"algoloji",
"klinik-psikolog-psikoloji-aile-danismani-psikolog",
"dahiliye-ve-ic-hastaliklari-endokrinoloji-ve-metabolizma-hastaliklari",
"spor-hekimligi",
"cocuk-sagligi-ve-hastaliklari-neonatoloji",
"akupunktur-biorezonans",
"dis-hekimi-agiz-dis-ve-cene-cerrahisi",
"kardiyoloji",
"endokrinoloji-ve-metabolizma-hastaliklari",
"medikal-estetik-sertifikali-tip-doktoru",
"genel-cerrahi",
"dis-hekimi-periodontoloji",
"plastik-rekonstruktif-ve-estetik-cerrahi-el-cerrahisi-ve-mikrocerrahi-plastik",
"kadin-hastaliklari-ve-dogum",
"dil-konusma-bozuklari-b",
"dis-hekimi-pedodonti",
"tibbi-onkoloji-dahiliye-ve-ic-hastaliklari",
"cocuk-sagligi-ve-hastaliklari-cocuk-gastroenterolojisi",
"cocuk-sagligi-ve-hastaliklari-cocuk-endokrinoloji",
"klinik-psikolog",
"tibbi-onkoloji",
"diyetisyen",
"radyoloji-uroradyoloji",
"dis-hekimi-oral-diagnoz-ve-radyoloji",
"tibbi-farmakoloji",
"anestezi-ve-reanimasyon",
"pedodonti",
"akupunktur",
"tibbi-genetik",
"noroloji",
"dahiliye-ve-ic-hastaliklari-immunoloji-ve-alerji-hastaliklari",
"plastik-rekonstruktif-ve-estetik-cerrahi",
"radyoloji",
"dis-hekimi-endodonti",
"goz-hastaliklari",
"gogus-cerrahisi",
"anestezi-ve-reanimasyon-algoloji-anestezi-ve-reanimasyon",
"psikoloji-aile-danismani-psikolog",
"kulak-burun-bogaz-hastaliklari",
"dahiliye-ve-ic-hastaliklari-geriatri",
"ortopedi-ve-travmatoloji-el-cerrahisi-ve-mikrocerrahi",
"pedagoji",
"gogus-hastaliklari",
"psikiyatri",
"beyin-ve-sinir-cerrahisi",
"dis-hekimi",
"gastroenteroloji",
"cocuk-psikiyatrisi",
"kalp-damar-cerrahisi",
"cocuk-sagligi-ve-hastaliklari-cocuk-norolojisi",
"anatomi",
"dil-konusma-bozuklari-b-aile-danismani",
"fiziksel-tip-ve-rehabilitasyon",
"psikoloji",
"noroloji-noroloji-algoloji",
"gogus-hastaliklari-alerji-ve-gogus-hastaliklari",
"psikoloji-psikoonkoloji",
"dis-hekimi-dis-protezi-uzmani",
"nukleer-tip"
]

specialities = sorted(specialities)

# Test

In [9]:
test_et(tokenizer, model, cihaz, titles, specialities)

Dropdown(description='Uzmanlık alanı:', options=('acil-tip', 'aile-danismani', 'aile-hekimligi', 'akupunktur',…

Uzmanlık alanı seçin ve Enter'a basın (Çıkış için '0' yazın): 



Dropdown(description='Doktor unvanı:', options=('Aile Danışmanı', 'Dil ve Konuşma Terapisti', 'Doç. Dr.', 'Doç…

Doktor unvanı seçin ve Enter'a basın (Çıkış için '0' yazın): 

Soru: 
hocam sağ gözümün çevresinde belirgin damar var bunun tedavisi için hangi bölüme muayane olmalıyım göz mü dermatoloji mi?


Dropdown(description='Uzmanlık alanı:', options=('acil-tip', 'aile-danismani', 'aile-hekimligi', 'akupunktur',…

Uzmanlık alanı seçin ve Enter'a basın (Çıkış için '0' yazın): 
0


# Örnekler

**Dt.
dis-hekimi
60 yaşında babamın dişlerini yaptırmak istiyoruz. yardımcı olabilir misiniz?
tabı ben patnostayım buyrun yardımcı olmak ısterım**

Dt.
dis-hekimi
bir aydır ağzımda geçmeyen ufak bir yara var alt çenemde protezdenmi acaba ne yapmalıyım kötü bir hastalık olur diye korkmaya başladım lütfen beni aydınlatırmısınız meliha özkan
protez travmasına baglı epulıs gelısebılır yada patolojık bi durum olabılır bulunudugunuz yerde fakulte varsa cene hastalıkları yada cerrahısıne gorunmenız onerılır

Dr. Dt.
dis-hekimi
Merhaba dun dis cekimi yapildiktan sonra bu sabah asiri kanama ile hekimime basvurdum.Pihtinin olustugunu yanliz kanamanin durmadigi gerekcesiyle dikis islemi uyguladi.Bugun herhangi bir kanamam veya agrim yok yalniz dikis bolgesinde mukusa benzer bir doku olustu.Tekrar gitmem gerekir mi?
Hayır.. rahat olun.normal şeyler yazmıssiniz.. yara yerinden mukus gibi dokunun cikmasi normaldir..


**Dr. Dt.
dis-hekimi
Merhaba.1,5 ay önce köprü dişlerim yapistirildi. O zaman ağrı vs yoktu.ancak son 10-15 gündür sanki bir şey içince köprünün altına gidiyor hissi var. Böyle düşünmeme sebep ise kendi disim ile köprü olan disin arasında diğer dişlere nazaran boşluk olması. Örneğin dış arası fırça kullanırken o bölgeye boydan boya girebiliyor fırça. Baslarda aralarda yapıştırıcı kaldığından hissetmedim belki de.ayrica köprü altındaki diğer dişlerimde kanal yapılanlarda da ağrı hissettim. Dişçi kontrol etti röntgen çekti herhangi bir sorun göremedi. Ayrıca baş ve kulak ağrısı var. 20lik dislerle de ilgili sıkıntı yok .daha önce dişciden kaynaklı bi sorundan dolayı güvenim azaldı dişçiye karşı. Dişçi emin bir şekilde koprunun altına gitmez dedi ama ben de bu sebeplerden dolayı emin olamadım.cevaplariniz için simdiden teşekkürler.
Öncelikle geçmiş olsun. Köprü yapımından sonra bu tarz hassasiyetler olabilir . Geçicidir. Kanallı dişlerinizde - kanalın yeni yapıldığını farz ederek söylüyorum- de ilk zamanlar, belki bir aya varan hassasiyet hissetmeniz normaldir. Röntgen çekildiğini ve sorun görülmediğini söylüyorsunuz. 20 liklerinizde de sorun görülmediğine göre belki baş kulak ağrısı için KBB ye görünmekte fayda olabilir diye düşünüyorum. Baş kulak ağrısı dişe bağlı olsa dişte ya da dişlerde hassasiyetten daha fazla ağrı şikayetiniz olurdu.**

Dr. Dt.
dis-hekimi
Merhaba doktor bey bnm çocuğumun üst dişlerinde 7ayligindan beri çürüme oluyor tamamen o dişinin beyaz yeri kalmaktadırlar curudu o kadar doktora götürdük ama bi tedavisini yapamadılar hepsi başka bişey diyo neden kaynaklanıyor olabilir
bazen lekeler çürük izlenimi verebiliyor.muayenede ortaya çıkar. uğrarsanız bilgilendirebiliriz...saygılarımızla...

**Dr. Dt.
dis-hekimi
Merhaba doktor bey bnm çocuğumun üst dişlerinde 7ayligindan beri çürüme oluyor tamamen o dişinin beyaz yeri kalmaktadırlar curudu o kadar doktora götürdük ama bi tedavisini yapamadılar hepsi başka bişey diyo neden kaynaklanıyor olabilir
ÖNCELİKLE GEÇMİŞ OLSUN BİR SÜRÜ SEBEBİ OLABİLİR ilaça bağlı renkleşme uzun süre biboron kullanımı çok fazla antibiyotik**